In [1]:
import json
import nltk
from nltk.corpus import stopwords
import re
from nltk import tokenize
from tqdm import tqdm
from nltk.corpus import words

In [2]:
st = stopwords.words('english')
common_words = words.words()

### Annotation dataset

Labels

1 - GENE

2 - DISEASE

In [3]:
with open("../genes.txt") as g:
    genes = set([ x.lower().strip() for x in g.readlines()])

In [4]:
st.append("ace")
st.append("large")
st.append("kit")
st.append("impact")
st.append("set")

In [5]:
genes_clean = []
for g in tqdm(genes):
    if g not in st and not g.isdigit() and g not in common_words:
        genes_clean.append(g)

100%|███████████████████████████████████████████████████████████████████████████| 16580/16580 [00:37<00:00, 437.84it/s]


In [6]:
len(genes_clean)

16278

In [7]:
len(genes_clean)

16278

In [8]:
with open("../Maladies.txt") as m:
    maladies = list(set([ x.lower().strip() for x in m.readlines()]))
    maladies.remove("disease")

Corpus

In [9]:
datas = open("../asthma.json").readlines()

### Create dataset by sentences

In [59]:
%%time
TRAIN_DATA_WORD=[]
train_i = 0
nb = 0
for line in tqdm(datas):
    text = json.loads(line)["ab"]
    train_i += 1
    
    detected_disease = []
    for maladie in maladies:
        if maladie in text:
            detected_disease.append(maladie.lower())
    
    for sentence in tokenize.sent_tokenize(text):
        if len(sentence) > 10:
            uhm = {"entities": []}
            
            for detect in detected_disease:
                if detect in sentence.lower():
                    pmz = [m.start() for m in re.finditer(r'\b%s\b' % re.escape(detect), sentence.lower())]
                    for qs in pmz:
                        uhm["entities"].append( (qs, qs+len(detect), "DISEASE")  )
            
            for n in tokenize.word_tokenize(sentence):
                if n.lower() in genes_clean:
                    pmz = [m.start() for m in re.finditer(r'\b%s\b' % re.escape(n.lower()), sentence.lower())]
                    for qs in pmz:
                        uhm["entities"].append( (qs, qs+len(n), "GENE")  )
            
            TRAIN_DATA_WORD.append( (sentence, uhm) )
    
    if train_i % 2000 == 0:
        with open("train/normal/train_set_"+str(nb)+".json", "w") as t:
            t.write(json.dumps(TRAIN_DATA_WORD))
        nb += 1
        TRAIN_DATA_WORD=[]

100%|████████████████████████████████████████████████████████████████████████████| 44975/44975 [41:47<00:00, 15.75it/s]


Wall time: 41min 47s


In [56]:
alld = []
allg = []
for dg in TRAIN_DATA_WORD[0:1000]:
    if dg[1]["entities"] :
        for d  in dg[1]["entities"]:
            if d[2] == "DISEASE":
                allg.append(dg[0][d[0]:d[1]])
            if d[2] == "GENE":
#                 print()
#                 print(dg[0][d[0]:d[1]], d[2])
#                 if dg[0][d[0]:d[1]] == "set":
#                     print(dg)
                alld.append(dg[0][d[0]:d[1]])

In [57]:
set(allg)

{'Asthma',
 'Atopic dermatitis',
 'Fibrous dysplasia',
 'Food allergy',
 'Herpes zoster',
 'Influenza',
 'Leukemia',
 'Obesity',
 'Osteoporosis',
 'allergic asthma',
 'allergic rhinitis',
 'asthma',
 'atopic dermatitis',
 'bronchiolitis',
 'cancer',
 'carcinoma',
 'chronic obstructive pulmonary disease',
 'coronary heart disease',
 'dermatitis',
 'diabetes mellitus',
 'egg allergy',
 'fibrous dysplasia',
 'food allergy',
 'gestational diabetes',
 'heart disease',
 'herpes zoster',
 'hypertension',
 'hypoglycemia',
 'influenza',
 'irritable bowel syndrome',
 'kidney disease',
 'leukemia',
 'lung cancer',
 'lung carcinoma',
 'lung disease',
 'lymphoma',
 'myasthenia gravis',
 'myocardial infarction',
 'myocarditis',
 'obesity',
 'obstructive lung disease',
 'optic atrophy',
 'osteoarthritis',
 'osteoporosis',
 'ovarian cancer',
 'pneumonia',
 'propionic acidemia',
 'psoriasis',
 'pulmonary fibrosis',
 'rhinitis',
 'sarcoidosis',
 'syndrome',
 'urticaria'}

In [58]:
set(alld)

{'APP',
 'BCL2',
 'CCL21',
 'CD4',
 'CHIT1',
 'CLPs',
 'Cs',
 'EED',
 'ERBB4',
 'FEV',
 'HR',
 'HTA',
 'IGF1R',
 'IQGAP2',
 'MyD88',
 'NHS',
 'NLRP3',
 'NMs',
 'NPs',
 'Ndst1',
 'OXR1',
 'RNASET2',
 'RhoA',
 'Sptlc2',
 'TGM3',
 'TLR2',
 'TLR7',
 'TLR8',
 'TNF',
 'TRAF6',
 'TSLP'}

In [ ]:
# %%time
# TRAIN_DATA_WORD=[]
# train_i = 0
# nb = 0
# for line in tqdm(datas[0:2]):
#     text = json.loads(line)["ab"].lower()
#     train_i += 1
    
#     detected_disease = []
#     for maladie in maladies:
#         if maladie in text:
#             detected_disease.append(k)
    
#     for sentence in tokenize.sent_tokenize(text):
#         if len(sentence) > 2:
#             uhm = {"entities": []}
            
#             for detect in detected_disease:
#                 if detect in sentence:
#                     pmz = [m.start() for m in re.finditer(r'\b%s\b' % re.escape(n2), sentence)]
#                     for qs in pmz:
#                         uhm["entities"].append( (qs, qs+len(n2), "DISEASE")  )
            
            
#             for n in sentence.split(" "):
#                 n2 = n.replace(".", "").replace(",", "")
#                 if n2 in maladies:
#                     pmz = [m.start() for m in re.finditer(r'\b%s\b' % re.escape(n2), sentence)]
#                     for qs in pmz:
#                         uhm["entities"].append( (qs, qs+len(n2), "DISEASE")  )
#                 if n2 in genes_clean:
#                     pmz = [m.start() for m in re.finditer(r'\b%s\b' % re.escape(n2), sentence)]
#                     for qs in pmz:
#                         uhm["entities"].append( (qs, qs+len(n2), "GENE")  )
            
            
            
#             TRAIN_DATA_WORD.append( (sentence, uhm) )
    
# #     if train_i % 2000 == 0:
# #         with open("train/train_set_"+str(nb)+".json", "w") as t:
# #             t.write(json.dumps(TRAIN_DATA_WORD))
# #         nb += 1
# #         TRAIN_DATA_WORD=[]